In [6]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy.optimize import *
import math
from math import sqrt

In [7]:
# The functions implement the theoretical developments in  
# "Hinczewski M, Schlagberger X, Rubinstein M, Krichevsky O, Netz RR. End-monomer dynamics in semiflexible polymers. Macromolecules. 2009 Feb 10;42(3):860-75."
# & Harnau L, Winkler RG, Reineker P. Dynamic structure factor of semiflexible macromolecules in dilute solution. The Journal of chemical physics. 1996 Apr 22;104(16):6355-68

def Afunc(n,L):
    return 16.06 - 12*np.log10(L/(a*pi*n)) 
  
def E1(z):
    def intg(t):
        return np.e**(-z*t)/t
    ing = sp.integrate.quad(intg,1,10000)
    return ing[0]

E1_15 = E1(3/2)

def I1nm(n,m,lp,L):
    if n < Llp:
        if n == m:
            return sqrt(6/pi)*(E1(6*a*a/lp/lp)-E1_15)
        else:
            return (4*sqrt(6/pi)/L)*(2*a*np.e**(-3/2) - lp*np.exp(-6*a*a/lp/lp) + a*sqrt(6*pi)*(sp.special.erf(sqrt(3/2)) - sp.special.erf(sqrt(6)*a/lp)))
    else:
        if n == m:
            return - (sqrt(6)/(n*(pi**1.5))) * (3 + n*pi*(3*gama + E1_15 + 2*np.log10(sqrt(6)*a*pi*n/L)))
        else:
            return (m**2 + n**2)*(np.log10(16*m*n*((a*pi/L)**2) + 2*gama)) - (2*sqrt(6)/(pi**1.5))*(m+n)*pi/(m**2+n**2)

def I2nm(n,m,lp,L):
    if n == m:
        return sqrt(6*L/(pi*lp))*(1/(n**0.5) - 1/(2*pi*(n**1.5))) - sqrt(48/pi)
    else:
        return - sqrt(24*L/lp)/((n+m)*(sqrt(n)+sqrt(m))*(pi**1.5))

def I1nm_nl(n,m,lp,L):
    alp = 6*((a/lp)**2)
    if n == m:
        return sqrt(6/pi)*(E1(alp)-E1_15)
    else:
        return (4*sqrt(6/pi)/L)*(2*a*np.e**(-3/2) - lp*np.e**(-alp) + a*sqrt(6*pi)*(sp.special.erf(sqrt(3/2)) - sp.special.erf(sqrt(6)*a/lp)))
    
def I1nm_ng(n,m,lp,L):
    if n == m:
        return - (sqrt(6)/(n*(pi**1.5))) * (3 + n*pi*(3*gama + E1_15 + 2*np.log10(sqrt(6)*a*pi*n/L)))
    else:
        return (m**2 + n**2)*(np.log10(16*m*n*((a*pi/L)**2) + 2*gama)) - (2*sqrt(6)/(pi**1.5))*(m+n)*pi/(m**2+n**2)
    
def MSD(rt,lp,tauD):
    
    summ = 0
    
    iFunc1 = 0
    iFunc2 = 0
    ilambdan = 0
    iHnn = 0
    iH0n = 0
    iEpsn = 0
    
    
    for i in range(1,itr):
        
        n = i
        
        s = L/2
        
        if n < Llp:
            iFunc1 = sqrt(L/lp)*((-108*pi + 27*(pi**2))/(18*sqrt(6)*(pi**(7/2))*(n**(3/2))) + (- 126 + 72*pi - 16*sqrt(3)*pi)/(18*sqrt(6)*(pi**(7/2))*(n**(5/2))))
            iFunc2 = sqrt(2/L)*((-9+4*sqrt(3))/18 + (-9+(9+sqrt(3))*pi)/(27*pi*pi*n) - (72 - 45*pi + 4*sqrt(3)*pi)/(216*(pi**3)*(n**2)))
            ilambdan = (3*KbT*(pi**2)*(n**2))/(2*lp*(L**2))
            iHnn = a*mu0*(2 + I1nm_nl(n=i,m=i,lp=lp,L=L) + I2nm(n=i,m=i,lp=lp,L=L))
            iH0n = a*mu0*(I1nm_nl(n=0,m=i,lp=lp,L=L) + I2nm(n=0,m=i,lp=lp,L=L))
            
            aln = pi*n/L
            if (n%2)==0:
                iEpsn = ((-1)**(n*0.5))*sqrt(2/L)*np.cos(aln*s)
            else:
                iEpsn = ((-1)**((n-1)*0.5))*sqrt(2/L)*np.sin(aln*s)
        
        else:
            iFunc1 = 12*(sqrt(6/pi))/(n*Afunc(n,L))
            iFunc2 = sqrt(4/L)/Afunc(n,L)
            ilambdan = 3*KbT*lp*((pi*(2*n-1)/L)**4)/32
            iHnn = a*mu0*(2 + I1nm(n=i,m=i,lp=lp,L=L))
            iH0n = a*mu0*I1nm(n=0,m=i,lp=lp,L=L)
            
            aln = pi*(2*n-1)/(2*L)
            if (n%2)==0:
                iEpsn = ((-1)**(n*0.5))*sqrt(2/L)*np.cos(aln*s) + sqrt(1/L)*np.cosh(aln*s)/np.sinh(aln*L/2)
            else:
                iEpsn = ((-1)**((n-1)*0.5))*sqrt(2/L)*np.sin(aln*s) + sqrt(1/L)*np.sinh(aln*s)/np.cosh(aln*L/2)
        
        
        itaun = 1/(ilambdan*iHnn) - iFunc1/(iHnn*iHnn*ilambdan)

        iEpsL2 = iEpsn + iH0n/(iHnn*(L**0.5)) + iFunc2
                
        iTheta = iHnn + 2 * iFunc1
        
        summ+= 6*KbT*itaun*iTheta*iEpsL2*iEpsL2*(1-np.exp(-rt/itaun))
    
    return 6*Diff_Hara(lp=lp,L=L)*rt*tauD + summ

# The calculation of diffusion proposed by "Harnau L, Winkler RG, Reineker P. Dynamic structure factor of semiflexible macromolecules in dilute solution. The Journal of chemical physics. 1996 Apr 22;104(16):6355-68"

def Diff_Hara(lp,L): 
    p = 0.5/lp
    d = 2*a
    def intg(s):
        As = s/p - (1-np.exp(-2*p*s))/(2*(p**2))
        return (L-s)*np.exp(-3*(d**2)/(2*As))/sqrt(As)
    ing = sp.integrate.quad(intg,d,L)
    return (C/(3*pi*L))*(1+sqrt(6/pi)*np.float(ing[0])/L)

def FCS(rt,lp,tauD):
    H = MSD(rt=rt,lp=lp,tauD=tauD)
    return 1/((1 + (2/3.0)*H/(width))* np.sqrt(1+(2/3.0)*H/(depth))) # FCS functional form for mean square displacement

AttributeError: module 'scipy' has no attribute 'integrate'

In [3]:
# Calls randomly slr (=20) curves from a lot of 180 curves, and fits the FCS function to the average of slr(=20) curves to obtain persistence length and diffusivity.

def runner(df,tcurve,slr):
    
    fda = np.zeros((5,tcurve))
    
    ltc =len(list(df))
    for j in range(tcurve):
        
        X1 = np.random.randint(low=1, high=ltc, size=(slr,))
        ll = df.iloc[:,X1]
        avgn = ll.mean(axis=1)
        yf = avgn/np.mean(avgn[4:20])
        
        popt,pcov = curve_fit(FCS,np.asarray(xf[st:en]),np.asarray(yf[st:en]),p0=[1.8e-07,0.5],bounds=([1e-8,0.],[1e-6,2.]),maxfev=10000)
        
        fda[0,j] = popt[0]
        fda[1,j] = Diff_Hara(popt[0],L)
        fda[2,j] = popt[-1]
        errr = np.sqrt(np.diag(pcov))
        fda[3,j] = errr[0]
        fda[4,j] = errr[1]
#         print(np.sqrt(np.diag(pcov)))

    return np.asarray(fda)

In [4]:
KbT    = (1.38e-23)*298
etta   = 8.9e-04
pi     = 3.14
C      = KbT/etta
a      = 1.0e-09
L      = 222e-09 
Llp    = 2
itr    = 6
gama   = 0.5772
mu0    = 1.0/(6*pi*etta*a)
amu0   = a*mu0
df     = pd.read_csv(r'EEA1.csv',encoding='utf-7')
SP     = 6.7
# width  = 4*71.66*299e-06*1e-12  #413e-12*41.7e-6*4*132/113     #w2n = Da*Ta*4*Tn/Td1
width = 413e-12*41.7e-6*4*132/113 
print(width)
depth  = SP*SP*width
st     = 60
en     = 280

xf     = df['t']

tcurve = 10  # bootstrapping sample size
name   = [r'GTP_EEA1.csv'] # Files other than GTP long term data, can be used here in this array

slr = 20

for j in range(len(name)):
    start  = time.perf_counter()

    df      = pd.read_csv(name[j],encoding='utf-7')
    emb     = runner(df,tcurve,slr)

    naam    = name[j][:-4]+'_'+str(tcurve)+'_'+str(slr)+'cal_L220'
    np.save(naam,emb)

    finish  = time.perf_counter()
    print(f'finished in {round(finish-start,2)} second(s)',naam)

8.047140530973451e-14


C:\Users\anupam\AppData\Local\Temp\ipykernel_19172\2012295705.py:122: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (C/(3*pi*L))*(1+sqrt(6/pi)*np.float(ing[0])/L)


finished in 3841.11 second(s) EEA1_100000_20cal_L220


C:\Users\anupam\AppData\Local\Temp\ipykernel_19172\2012295705.py:122: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (C/(3*pi*L))*(1+sqrt(6/pi)*np.float(ing[0])/L)


finished in 3496.56 second(s) EEA1_2uMGST_100000_20cal_L220


C:\Users\anupam\AppData\Local\Temp\ipykernel_19172\2012295705.py:122: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return (C/(3*pi*L))*(1+sqrt(6/pi)*np.float(ing[0])/L)


finished in 3612.15 second(s) EEA1_2uMRab5GTPyS_100000_20cal_L220
